In [0]:
pip install json5

Python interpreter will be restarted.
Collecting json5
 Downloading json5-0.9.5-py2.py3-none-any.whl (17 kB)
Installing collected packages: json5
Successfully installed json5-0.9.5
Python interpreter will be restarted.

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka.ca");

res2: String = dbfs:/FileStore/tables/cloudkarafka.ca

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka.jaas")


auth.lang:dbfs:/FileStore/tables/cloudkarafka.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka.jaas

In [0]:
dbutils.fs.mkdirs("/FileStore/tables/stream/input/transactions")

Out[1]: True

In [0]:
%fs rm -r /tmp/stream/folder2kafka/*

res1: Boolean = false

In [0]:
from pyspark.sql.types import *
from pyspark.sql.types import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import json5 as json
from pyspark.sql.functions import col

kfk_topic_out = "yrmfqh3q-transactions-input" #remember to create in cloudkarfka
kfk_brokers = "omnibus-01.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094"
sasl_protocol = "SASL_SSL"
sasl_mech = "SCRAM-SHA-256"
sasl_username = "yrmfqh3q"
sasl_password = "WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc"

source_folder = "/FileStore/tables/stream/input/transactions/"
#t_id,t_timestamp,t_user,t_item,t_quantity
#T_ID,T_TS,T_U,T_I,T_Q

trans_struct = StructType([ StructField("t_id", StringType(), True), StructField("t_timestamp", StringType(), True), StructField("t_user", StringType(), True),StructField("t_item", StringType(), True), StructField("t_quantity", StringType(), True) ])
#options(inferSchema='true') - since we are specifying schema(trans_struct)
print("trans_struct:", trans_struct)
trans_df = spark.readStream.options(header='true').csv(source_folder, schema = trans_struct)
print("trans_df(1):", type(trans_df), trans_df)
trans_df = trans_df.selectExpr("to_json(struct(*)) AS value")
print("trans_df(2):", type(trans_df), trans_df)
trans_df.writeStream.format("kafka")\
        .option("checkpointLocation", "/tmp/stream/folder2kafka/")\
        .option("kafka.bootstrap.servers", kfk_brokers)\
        .option("kafka.security.protocol", sasl_protocol) \
        .option("kafka.sasl.mechanism", sasl_mech) \
        .option("security.protocol", sasl_protocol) \
        .option("sasl.mechanisms", sasl_mech) \
        .option("topic", kfk_topic_out)\
        .start()\
        .awaitTermination()

trans_struct: StructType(List(StructField(t_id,StringType,true),StructField(t_timestamp,StringType,true),StructField(t_user,StringType,true),StructField(t_item,StringType,true),StructField(t_quantity,StringType,true)))
trans_df(1): <class 'pyspark.sql.dataframe.DataFrame'> DataFrame[t_id: string, t_timestamp: string, t_user: string, t_item: string, t_quantity: string]
trans_df(2): <class 'pyspark.sql.dataframe.DataFrame'> DataFrame[value: string]